In [153]:
import os
from os import path
import glob
import pandas as pd
import json
from collections import defaultdict

In [154]:
perf_dir = "/share/data/speech/shtoshni/research/events/models/perf/"
slurm_id = "6407273"

files = sorted(glob.glob(path.join(perf_dir, slurm_id + "*")), key=lambda x: int(path.splitext(path.basename(x))[0].split('_')[1]))
print(len(files))

20


In [155]:
model_dict_list = []
for file in files:
    model_dict_list.append(json.loads(open(file).read()))

In [156]:
def determine_varying_attributes(model_dict_list, ignore_attribs=['model_dir', 'train', 'test', 'dev', 'pretrained_mention_model',
                                                                  'slurm_id', 'best_model_dir', 'data_dir']):
    attrib_to_vals = defaultdict(set)
    for model_dict in model_dict_list:
        for attrib, val in model_dict.items():
            if attrib in ignore_attribs:
                continue
            else:
                attrib_to_vals[attrib].add(val)
    
    
    varying_attribs = []
    for attrib, vals in attrib_to_vals.items():
        if len(vals) > 1:
            varying_attribs.append(attrib)
            
    return varying_attribs

In [157]:
varying_attribs = determine_varying_attributes(model_dict_list)


In [158]:
perf_df = pd.DataFrame(columns=(varying_attribs))

for model_dict in model_dict_list:
    perf_dict = {}
    for attrib in varying_attribs:
        perf_dict[attrib] = model_dict[attrib]
            
    perf_dict['fscore'] = round(model_dict['dev']['fscore'], 1)
    perf_dict['test_f'] = round(model_dict['test']['fscore'], 1)
                
    perf_df = perf_df.append(perf_dict, ignore_index=True)

In [159]:
print(perf_df.to_markdown()) 


|    | use_local_attention   |   mlp_size | use_srl   |   sample_invalid |   srl_loss_wt |   fscore |   test_f |
|---:|:----------------------|-----------:|:----------|-----------------:|--------------:|---------:|---------:|
|  0 | False                 |       1000 | False     |             0.25 |          1    |     47   |     48.7 |
|  1 | True                  |       1000 | False     |             0.25 |          1    |     47.9 |     49.4 |
|  2 | False                 |       1000 | True      |             0.25 |          0.01 |     48.6 |     48.8 |
|  3 | False                 |       1000 | True      |             0.25 |          0.1  |     47.1 |     48.9 |
|  4 | False                 |       1000 | True      |             0.25 |          1    |     47.3 |     48.4 |
|  5 | False                 |       1000 | False     |             0.5  |          1    |     48.7 |     49.9 |
|  6 | True                  |       1000 | False     |             0.5  |          1    |     4

In [160]:
print(perf_df.to_csv()) 


,use_local_attention,mlp_size,use_srl,sample_invalid,srl_loss_wt,fscore,test_f
0,False,1000,False,0.25,1.0,47.0,48.7
1,True,1000,False,0.25,1.0,47.9,49.4
2,False,1000,True,0.25,0.01,48.6,48.8
3,False,1000,True,0.25,0.1,47.1,48.9
4,False,1000,True,0.25,1.0,47.3,48.4
5,False,1000,False,0.5,1.0,48.7,49.9
6,True,1000,False,0.5,1.0,47.0,46.8
7,False,1000,True,0.5,0.01,46.8,48.6
8,False,1000,True,0.5,0.1,47.3,49.2
9,False,1000,True,0.5,1.0,46.7,49.1
10,False,3000,False,0.25,1.0,48.7,48.8
11,True,3000,False,0.25,1.0,47.9,48.8
12,False,3000,True,0.25,0.01,47.1,50.1
13,False,3000,True,0.25,0.1,48.9,49.3
14,False,3000,True,0.25,1.0,48.4,48.3
15,False,3000,False,0.5,1.0,47.2,49.0
16,True,3000,False,0.5,1.0,47.3,48.0
17,False,3000,True,0.5,0.01,47.3,49.1
18,False,3000,True,0.5,0.1,48.7,48.7
19,False,3000,True,0.5,1.0,47.6,49.3

